In [2]:
import json
import requests
import os.path
import time

import numpy as np
import pandas as pd
from string import digits, punctuation

# neo4j credentials
SERVER: neo4j+s://f1329174.databases.neo4j.io:7687

SERVER_NAME: linkedin_github

USERNAME:neo4j

PASSWORD:3BvD2pXSsyx5Wp3sONMnSj02oFytmekldc0hW8x8SwI

In [37]:
de_github = pd.read_excel('de_with_github.xlsx')
ds_github = pd.read_excel('ds_with_github.xlsx')

In [40]:
ds_github.head(2).T

,0,1
Unnamed: 0,1,2
linkedinProfile,https://www.linkedin.com/in/tom-haxton-4a231b89/,https://www.linkedin.com/in/navata-m/
fullName,Tom Haxton,Navata M.
github_urls,https://github.com/tkhaxton,NaN
description,I'm a data science manager at Quora. I manage ...,"\nSkills:\nMachine Learning, Statistical Model..."
company,Quora,LendingClub
jobTitle,Data Science Manager,Data Scientist
location,San Francisco Bay Area,San Francisco Bay Area
school,University of Pennsylvania,"Galvanize - San Francisco, SoMa"
schoolDegree,Doctor of Philosophy (PhD),Data Science Immersive


In [14]:
to_keep = ['fullName', 'description', 'company', 'jobTitle', 'location', 'school',
       'schoolDegree', 'allSkills', 'github_urls']

In [15]:
df = pd.concat([ds_github, de_github], ignore_index=True)
df = df[to_keep]

In [31]:
# save github urls to csv
# git_urls = df[['github_urls']]
#git_urls.to_csv('git_urls.csv')

In [7]:
# capitalize first letter
df = df.applymap(lambda s: s.title() if type(s) == str else s)

In [16]:
# clean up text based data
# remove punctuation
df['description'] = df['description'].apply(lambda x: x.translate(str.maketrans('', '', punctuation)))
# remove digits
df['description'] = df['description'].apply(lambda x: x.translate(str.maketrans('', '', digits)))
# lowercase strings
df['description'] = df['description'].apply(lambda x: x.lower())

# replace "\n" in description
df['description'] = df['description'].apply(lambda x: x.replace("\n", " "))

In [17]:
# make a list for all the skills
df['allSkills'] = df['allSkills'].apply(lambda x: x.split(','))

In [18]:
df['allSkills']

0      [Data Analysis,  Research,  High Performance C...
1      [Data Analysis,  SQL,  Machine Learning,  Data...
2      [Computer Science,  Java,  C++,  Big Data,  Ba...
3      [Management,  Data Science,  Statistics,  Anal...
4      [Python (Programming Language),  Data Analytic...
                             ...                        
447    [Python (Programming Language),  PySpark,  Had...
448    [Scala,  Amazon Web Services (AWS),  Apache Sp...
449    [Python,  SQL,  Machine Learning,  Adobe Illus...
450    [Machine Learning,  Python,  Data Analysis,  D...
451    [Python,  Machine Learning,  SQL,  Physics,  D...
Name: allSkills, Length: 452, dtype: object

In [19]:
# explode 'allSkills' columns
df = df.explode('allSkills')
df.index.names = ['profileID']

In [20]:
df

,fullName,description,company,jobTitle,location,school,schoolDegree,allSkills,github_urls
profileID,,,,,,,,,
0,Tom Haxton,im a data science manager at quora i manage a ...,Quora,Data Science Manager,San Francisco Bay Area,University of Pennsylvania,Doctor of Philosophy (PhD),Data Analysis,https://github.com/tkhaxton
0,Tom Haxton,im a data science manager at quora i manage a ...,Quora,Data Science Manager,San Francisco Bay Area,University of Pennsylvania,Doctor of Philosophy (PhD),Research,https://github.com/tkhaxton
0,Tom Haxton,im a data science manager at quora i manage a ...,Quora,Data Science Manager,San Francisco Bay Area,University of Pennsylvania,Doctor of Philosophy (PhD),High Performance Computing,https://github.com/tkhaxton
0,Tom Haxton,im a data science manager at quora i manage a ...,Quora,Data Science Manager,San Francisco Bay Area,University of Pennsylvania,Doctor of Philosophy (PhD),Statistical Modeling,https://github.com/tkhaxton
0,Tom Haxton,im a data science manager at quora i manage a ...,Quora,Data Science Manager,San Francisco Bay Area,University of Pennsylvania,Doctor of Philosophy (PhD),Scientific Writing,https://github.com/tkhaxton
...,...,...,...,...,...,...,...,...,...
451,Jeremy Chow,passionate about deploying applications to mak...,TEKsystems @ Facebook,Data Engineer,San Francisco Bay Area,UC Berkeley College of Engineering,Bachelor of Science (BS),Recommender Systems,https://github.com/jeremyrchow
451,Jeremy Chow,passionate about deploying applications to mak...,TEKsystems @ Facebook,Data Engineer,San Francisco Bay Area,UC Berkeley College of Engineering,Bachelor of Science (BS),Time Series Analysis,https://github.com/jeremyrchow
451,Jeremy Chow,passionate about deploying applications to mak...,TEKsystems @ Facebook,Data Engineer,San Francisco Bay Area,UC Berkeley College of Engineering,Bachelor of Science (BS),Kubernetes,https://github.com/jeremyrchow


In [160]:
company_df = pd.DataFrame(pd.unique(df['company']), columns = ['company'])
company_df.index.names = ['companyID']
company_df['company'] = company_df['company'].apply(lambda x: x.translate(str.maketrans('', '', punctuation)))
company_df['company'][97] = 'Liba Edu'
#company_df.to_csv('company.csv')

In [143]:
school_df = pd.DataFrame(pd.unique(df['school']), columns = ['school'])
school_df.index.names = ['schoolID']
#school_df.to_csv('school.csv')

In [144]:
skill_df = pd.DataFrame(pd.unique(df['allSkills']), columns = ['skill'])
skill_df.index.names = ['skillID']
#skill_df.to_csv('skill.csv')

In [146]:
title_df = pd.DataFrame(pd.unique(df['jobTitle']), columns = ['title'])
title_df.index.names = ['titleID']
#title_df.to_csv('title.csv')

In [152]:
location_df = pd.DataFrame(pd.unique(df['location']), columns = ['location'])
location_df.index.names = ['locationID']
#location_df.to_csv('location.csv')

In [148]:
degree_df = pd.DataFrame(pd.unique(df['schoolDegree']), columns = ['degree'])
degree_df.index.names = ['degreeID']
#degree_df.to_csv('degree.csv')

In [27]:
#df.to_csv('profile.csv')

In [56]:
company_df = pd.read_csv('company.csv')
degree_df = pd.read_csv('degree.csv')
location_df = pd.read_csv('location.csv')
skill_df = pd.read_csv('skill.csv')
school_df = pd.read_csv('school.csv')
title_df = pd.read_csv('title.csv')

profile_df = pd.read_csv('profile.csv')

In [21]:
df['github_urls'].fillna("not available", inplace = True)
df['token'] = df['github_urls'].apply(lambda x: os.path.split(x)[1])

In [31]:
print(df.shape)
df.head()

(15979, 10)


,fullName,description,company,jobTitle,location,school,schoolDegree,allSkills,github_urls,token
profileID,,,,,,,,,,
0,Tom Haxton,im a data science manager at quora i manage a ...,Quora,Data Science Manager,San Francisco Bay Area,University of Pennsylvania,Doctor of Philosophy (PhD),Data Analysis,https://github.com/tkhaxton,tkhaxton
0,Tom Haxton,im a data science manager at quora i manage a ...,Quora,Data Science Manager,San Francisco Bay Area,University of Pennsylvania,Doctor of Philosophy (PhD),Research,https://github.com/tkhaxton,tkhaxton
0,Tom Haxton,im a data science manager at quora i manage a ...,Quora,Data Science Manager,San Francisco Bay Area,University of Pennsylvania,Doctor of Philosophy (PhD),High Performance Computing,https://github.com/tkhaxton,tkhaxton
0,Tom Haxton,im a data science manager at quora i manage a ...,Quora,Data Science Manager,San Francisco Bay Area,University of Pennsylvania,Doctor of Philosophy (PhD),Statistical Modeling,https://github.com/tkhaxton,tkhaxton
0,Tom Haxton,im a data science manager at quora i manage a ...,Quora,Data Science Manager,San Francisco Bay Area,University of Pennsylvania,Doctor of Philosophy (PhD),Scientific Writing,https://github.com/tkhaxton,tkhaxton


In [32]:
repo_df = pd.read_csv('repo.csv')
repo_df.rename(columns={'description':'repoDescription'}, inplace=True)

In [35]:
df = df.merge(repo_df, how='outer', on='token')

In [36]:
#df.to_csv('profile_with_repo.csv')